<a href="https://colab.research.google.com/github/douglascosta01/Data-Mining-Mnera-o-de-Dados-/blob/main/projeto_ml_dados_covid.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Pacotes

In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import sklearn.datasets
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import seaborn as sns
import matplotlib.pyplot as plt

Carga de dados

In [8]:
import pandas as pd

url = 'https://www.saopaulo.sp.gov.br/wp-content/uploads/2023/05/20230522_vacinometro.csv'

df = pd.read_csv(url)

print(df.head())


  MUNICÍPIO;1 DOSE1;1 DOSE2;2 DOSE;3 DOSE;REFORCO BIVALENTE;1 REFORCO;2 REFORCO;3 REFORCO;DOSE UNICA;ADICIONAL
0  ADAMANTINA;32370;32370;30845;104;2918;22480;94...                                                          
1     ADOLFO;4404;4404;4310;10;766;3668;2028;7;76;18                                                          
2  AGUAÍ;32215;32215;30436;50;4020;21783;11053;36...                                                          
3  AGUDOS;36574;36574;33721;52;1714;21447;8748;60...                                                          
4  ALAMBARI;5434;5434;5213;12;657;3805;1905;13;143;7                                                          


In [10]:
import pandas as pd
from IPython.display import display

url = 'https://www.saopaulo.sp.gov.br/wp-content/uploads/2023/05/20230522_vacinometro.csv'

df = pd.read_csv(url)

display(df)


,MUNICÍPIO;1 DOSE1;1 DOSE2;2 DOSE;3 DOSE;REFORCO BIVALENTE;1 REFORCO;2 REFORCO;3 REFORCO;DOSE UNICA;ADICIONAL
0,ADAMANTINA;32370;32370;30845;104;2918;22480;94...
1,ADOLFO;4404;4404;4310;10;766;3668;2028;7;76;18
2,AGUAÍ;32215;32215;30436;50;4020;21783;11053;36...
3,AGUDOS;36574;36574;33721;52;1714;21447;8748;60...
4,ALAMBARI;5434;5434;5213;12;657;3805;1905;13;143;7
...,...
641,ÁGUAS DE SÃO PEDRO;3886;3886;3810;11;591;3264;...
642,ÁLVARES FLORENCE;3763;3763;3692;10;548;2757;11...
643,ÁLVARES MACHADO;22747;22747;21445;38;2176;1503...
644,ÁLVARO DE CARVALHO;6172;6172;6130;11;1496;5399...


Separação dos Atributos

In [ ]:
import pandas as pd

# Ler os dados do DataFrame a partir da URL
url = 'https://www.saopaulo.sp.gov.br/wp-content/uploads/2023/05/20230522_vacinometro.csv'
df = pd.read_csv(url)

# Separar os atributos
df['Atributos'] = df['MUNICÍPIO;1 DOSE1;1 DOSE2;2 DOSE;3 DOSE;REFORCO BIVALENTE;1 REFORCO;2 REFORCO;3 REFORCO;DOSE UNICA;ADICIONAL'].str.split(';')

# Exibir os atributos
for atributos in df['Atributos']:
    print(atributos)


Carga dos dados populacionais de São Paulo

In [26]:
!pip install pandas
!pip install xlrd


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [27]:
import pandas as pd

# URL do arquivo Excel
url = 'https://ftp.ibge.gov.br/Censos/Censo_Demografico_2022/Previa_da_Populacao/POP2022_Municipios.xls'

# Carregar o arquivo Excel em um DataFrame
df = pd.read_excel(url, engine='xlrd')

# Exibir o DataFrame
print(df.head())


  Prévia da população calculada com base nos resultados do Censo Demográfico 2022 até 25 de dezembro de 2022  \
0                                                 UF                                                           
1                                                 RO                                                           
2                                                 RO                                                           
3                                                 RO                                                           
4                                                 RO                                                           

  Unnamed: 1  Unnamed: 2             Unnamed: 3 Unnamed: 4  
0    COD. UF  COD. MUNIC      NOME DO MUNICÍPIO  POPULAÇÃO  
1         11       00015  Alta Floresta D'Oeste      21558  
2         11       00023              Ariquemes     100896  
3         11       00031                 Cabixi       5107  
4         11       000

Separação dos dados demográficos de São Paulo

In [32]:
import pandas as pd

# Carregar o arquivo Excel em um DataFrame, pulando as primeiras 2 linhas de cabeçalho
df = pd.read_excel('https://ftp.ibge.gov.br/Censos/Censo_Demografico_2022/Previa_da_Populacao/POP2022_Municipios.xls', header=None, skiprows=2)

# Filtrar apenas os municípios de São Paulo
df_sp = df[df[0] == 'SP']

# Definir os nomes das colunas
df_sp.columns = df_sp.iloc[0]

# Remover a primeira linha (cabeçalho duplicado)
df_sp = df_sp.iloc[1:]

# Exibir o DataFrame com os municípios de São Paulo
print(df_sp)


3267  SP  35.0    105.0              Adamantina   34357
3268  SP  35.0    204.0                  Adolfo    4339
3269  SP  35.0    303.0                   Aguaí   31561
3270  SP  35.0    402.0          Águas da Prata    7075
3271  SP  35.0    501.0        Águas de Lindóia   18859
3272  SP  35.0    550.0  Águas de Santa Bárbara    7118
...   ..   ...      ...                     ...     ...
3907  SP  35.0  57006.0              Votorantim  137319
3908  SP  35.0  57105.0             Votuporanga   96795
3909  SP  35.0  57154.0                Zacarias    2680
3910  SP  35.0  57204.0               Chavantes   12090
3911  SP  35.0  57303.0            Estiva Gerbi   11394

[644 rows x 5 columns]


In [33]:

import pandas as pd

url_populacao = 'https://www.saopaulo.sp.gov.br/wp-content/uploads/2023/05/20230522_vacinometro.csv'
df_populacao = pd.read_csv(url_populacao)
